In [ ]:
!pip install zeep

In [ ]:
# Testovani operaci WSDP - tahani z katastru nemovitosti
import os
from requests import Session
from zeep import Client
from zeep.transports import Transport

# 1) URL na WSDL („Informace o bytech a nebytových prostorech“ v2.9)
WSDL_URL = "https://katastr.cuzk.cz/dokumentace/ws29/wsdp/informace_v29.wsdl"

# 2) (Ne)autentizace – když máte účet pro WSDP, nastavte CUZK_USER / CUZK_PASS
session = Session()
user = os.getenv("CUZK_USER")
pw   = os.getenv("CUZK_PASS")
if user and pw:
    session.auth = (user, pw)
transport = Transport(session=session)

# 3) Vytvoříme SOAP-klienta
client = Client(wsdl=WSDL_URL, transport=transport)

# 4) Vybereme první service a první port
service = next(iter(client.wsdl.services.values()))
port    = next(iter(service.ports.values()))

# 5) Vypíšeme všechny operace a jejich parametry
print("=== Seznam operací a jejich vstupních parametrů ===\n")
for op_name, oper in port.binding._operations.items():
    params = [el[0] for el in oper.input.body.type.elements]
    print(f"- {op_name}: {params}")
print("\n=== Konec seznamu ===")


=== Seznam operací a jejich vstupních parametrů ===

- stavWS: ['_value_1']
- dejMBRParcel: ['parcelaId']
- dejInfoOJednotce: ['idJednotky', 'castObceKod', 'stavbaTypKod', 'stavbaCislo', 'jednotkaCislo']
- dejNahledLV: ['katuzeKod', 'lvCislo', 'lvId']

=== Konec seznamu ===


In [ ]:
# Tahání informací k bytům - evidence dle zákona o vlastnictví bytů (ZBV) a podle občanského zákoníku (NOZ)
# vyžaduje účet a je zpoplatněno 


import os
from requests import Session
from zeep import Client
from zeep.transports import Transport
from zeep.wsse.username import UsernameToken

# === 1) Nastavení přístupu ===
# WSDL pro Informace o bytech a nebytových prostorech (v2.9)
WSDL_URL = "https://katastr.cuzk.cz/dokumentace/ws29/wsdp/informace_v29.wsdl"

# Přihlášení do WSDP (nutné pro WS-Security)
user   = os.getenv("CUZK_USER", "VAŠE_UŽIV_JMÉNO")
passwd = os.getenv("CUZK_PASS", "VAŠE_HESLO")

# HTTP transport (použijeme ale WSSE pro autentizaci)
session = Session()
transport = Transport(session=session)

# === 2) Vytvoření Zeep klienta s WS-Security UsernameToken ===
client = Client(
    wsdl=WSDL_URL,
    transport=transport,
    wsse=UsernameToken(user, passwd)
)

# === 3) Hard-coded testovací hodnoty ===
castObceKod   = "755541"   # kód k.ú.
stavbaTypKod  = "P"        # 'P'=popisné / 'E'=evidenční
stavbaCislo   = "1817"     # číslo budovy
jednotkaCislo = "36"       # číslo jednotky

# === 4) Volání operace dejInfoOJednotce ===
response = client.service.dejInfoOJednotce(
    castObceKod   = castObceKod,
    stavbaTypKod  = stavbaTypKod,
    stavbaCislo   = stavbaCislo,
    jednotkaCislo = jednotkaCislo
)

# === 5) Vytažení zakladJednotky ===
basis = getattr(response, "zakladJednotky", None) or getattr(response, "ZakladJednotky", None)

if basis == "ZBV":
    print("✔ Evidováno podle zákona o vlastnictví bytů (ZBV).")
elif basis == "NOZ":
    print("✔ Evidováno podle občanského zákoníku (NOZ).")
else:
    print(f"⚠ Neočekávaná hodnota zakladJednotky: {basis!r}")


Fault: Error on verifying message against security policy Error code:1000